In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else)","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc)",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist)",Name of the institute I graduated from last.,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you),Submitting this document means giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.,My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Video Introduction
0,Male,32,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,Certified financial analyst CFA,Masters,Health concious due to health conditions in fa...,I give consent,,,,,,
1,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,Institute of chartered accountants of India,Masters,I like playing sports,I give consent,,,,,,
2,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),"RRCE, Bangalore",Masters,No,I give consent,,,,,,
3,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,PGI Rohtak,Phd and equivalent,I take time to open up and quite curious,I give consent,,,,,,
4,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,XIC,PG Diploma,I believe honest communication and mutual resp...,I give consent,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,Female,31,Ghaziabad,,"Working , supportive, trustworthy, understand...",It was challenging,Never married,No,Under 12 Lakhs,IT,Bishop patrick school,Masters,,I give consent,Hindu,I am not sure,It will not work for me,Yes,Instagram,
299,Male,39,Pune,,"Having a supportive family, dependable friends...",I had a few of my own challenges in life which...,Was married earlier,No,Between 40-50 Lakhs,Experience Designer in digital space,"Vaishnav Institute of S&T, Indore",Undergraduate,"I have a cat, hope you don't afraid of animals",I give consent,Spiritual,Yes,It will not work for me,Yes,Instagram,
300,Male,36,Hyderabad,,Having a support family and frnds,Friends and memories,"Separated, not legally divorced",Yes,Between 20-24 Lakhs,Business,Osmania,Masters,Good relationship,I give consent,Hindu,Yes,Yes I am,Yes,Instagram,
301,Male,45,Bangalore,,Dependable friends,Awesome,Was married earlier,No,Between 40-50 Lakhs,Marketing,nihrd,Masters,No,I give consent,Hindu,Yes,Yes I am,Yes,Linkedin,


In [3]:
dataset.index = np.arange(2, len(dataset) + 2)
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else)","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc)",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist)",Name of the institute I graduated from last.,Educational degree,Anything else that you may like to mention to the community members (any specifics which may be important to you),Submitting this document means giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.,My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Video Introduction
2,Male,32,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...","Separated, filed for a divorce",No,Between 40-50 Lakhs,Finance Professional,Certified financial analyst CFA,Masters,Health concious due to health conditions in fa...,I give consent,,,,,,
3,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,Institute of chartered accountants of India,Masters,I like playing sports,I give consent,,,,,,
4,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),"RRCE, Bangalore",Masters,No,I give consent,,,,,,
5,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,PGI Rohtak,Phd and equivalent,I take time to open up and quite curious,I give consent,,,,,,
6,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,XIC,PG Diploma,I believe honest communication and mutual resp...,I give consent,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,Female,31,Ghaziabad,,"Working , supportive, trustworthy, understand...",It was challenging,Never married,No,Under 12 Lakhs,IT,Bishop patrick school,Masters,,I give consent,Hindu,I am not sure,It will not work for me,Yes,Instagram,
301,Male,39,Pune,,"Having a supportive family, dependable friends...",I had a few of my own challenges in life which...,Was married earlier,No,Between 40-50 Lakhs,Experience Designer in digital space,"Vaishnav Institute of S&T, Indore",Undergraduate,"I have a cat, hope you don't afraid of animals",I give consent,Spiritual,Yes,It will not work for me,Yes,Instagram,
302,Male,36,Hyderabad,,Having a support family and frnds,Friends and memories,"Separated, not legally divorced",Yes,Between 20-24 Lakhs,Business,Osmania,Masters,Good relationship,I give consent,Hindu,Yes,Yes I am,Yes,Instagram,
303,Male,45,Bangalore,,Dependable friends,Awesome,Was married earlier,No,Between 40-50 Lakhs,Marketing,nihrd,Masters,No,I give consent,Hindu,Yes,Yes I am,Yes,Linkedin,


In [4]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

In [5]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
300,Female,31,Ghaziabad,Never married
301,Male,39,Pune,Was married earlier
302,Male,36,Hyderabad,"Separated, not legally divorced"
303,Male,45,Bangalore,Was married earlier


In [6]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [8]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [9]:
encoded_categorical_features

,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,Widow/ Widower
2,0,1,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0
5,0,0,0,1,0
6,1,0,0,0,0
...,...,...,...,...,...
300,1,0,0,0,0
301,0,0,0,1,0
302,0,0,1,0,0
303,0,0,0,1,0


In [10]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [11]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
300,Female,31,Ghaziabad,Never married
301,Male,39,Pune,Was married earlier
302,Male,36,Hyderabad,"Separated, not legally divorced"
303,Male,45,Bangalore,Was married earlier


In [12]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [13]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,32,Mumbai,"Separated, filed for a divorce"
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
300,Female,31,Ghaziabad,Never married
301,Male,39,Pune,Was married earlier
302,Male,36,Hyderabad,"Separated, not legally divorced"
303,Male,45,Bangalore,Was married earlier


In [14]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.70710678]]


In [15]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [16]:
print(features.shape)

(303, 137)


In [17]:
k =280
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=280)

In [18]:
row =210
selectedRow = refDataset.iloc[row]

In [19]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[210 220 113 109 252  11 108 272 165  45 266  25  41 278 245 247 261 290
   48 156  24  26 229  81 139 256 181 288  51 123  54 110 122  30 262 118
   97  84 136 128 295  86  19 231 260 259  55 274 114  75  42 183 265 287
  200  52 100  32 185  73  50  18 105  29  12 111 251 258  65   1  27 174
  301 178 228   6  56 244 282 291 246 221  96 296  77  66 121 166 169 237
   36 196 135  83 201 209 160 106 194 241  89 143  91  71 219 208 171 163
  255 184 230 238 271  21 112 104 264 107 205 119 142 147 263 173 214 211
  249  53  35  76  62  37  59  79  94 137  70 191 242 141 180  23  61 297
  101 152 257 188   9  13 279  43 250 126  88 179 131   8 276  57 117 293
   60  95 269 134 213 234 299 216 124  33  22  10  14  46 223 146 217 130
  132 133 236 199 189 170 195 212 285  72 294 222  39 138 127 192 145  16
  292 154 253 273 153  49  85  40 233 232 225  38  44 102 218 150 204 116
  270  80 161 190 103 140 129   2 167 284 203  69 182   4 267 239 155  17
   63 226 268 151  68  28  67  64  92 

In [20]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Male' 43 'Delhi' 'Never married']


['Male' 54 'Delhi ' 'Was married earlier']


['Male' 45 'Delhi ' 'Was married earlier']


['Male' 44 'Delhi' 'Was married earlier']


['Male' 37 'Delhi' 'Never married']


['Male' 49 'New Delhi ' 'Was married earlier']


['Male' 43 'Delhi' 'Was married earlier']


['Male' 42 'Mumbai' 'Never married']


['Male' 42 'Bangalore ' 'Never married']


['Male' 36 'Delhi' 'Never married']


['Male' 36 'Delhi ' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 45 'New Delhi ' 'Was married earlier']


['Male' 50 'Bangalore' 'Widow/ Widower']


['Male' 40 'Hyderabad/pune' 'Never married']


['Male' 35 'Delhi' 'Never married']


['Male' 48 'Mumbai ' 'Separated, filed for a divorce']


['Male' 48 'Bangalore' 'Separated, not legally divorced']


['Male' 46 'Greater Noida' 'Separated, not legally divorced']


['Male' 39 'Mumbai' 'Never married']


['Male' 39 'Gurgaon' 'Never married']


['Male' 39 'Banga

In [21]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [22]:
print(preprocessData(refDataset))

[[ 1.         -0.90063572  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.20536789  1.         ...  0.          0.
   0.        ]
 [ 1.         -0.90063572  1.         ...  0.          0.
   0.        ]
 ...
 [ 1.         -0.20536789  0.         ...  0.          0.
   0.        ]
 [ 1.          1.35898472  0.         ...  0.          0.
   0.        ]
 [ 0.         -1.42208658  1.         ...  0.          0.
   0.70710678]]


In [23]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']